In [5]:
import threading
import time


def print_time(threadName, delay, iterations):
    start = int(time.time())
    for i in range(0,iterations):
        time.sleep(delay)
        print(f'{int(time.time() - start)} - {threadName}')

threads = [
    threading.Thread(target=print_time, args=('Fizz', 3, 6)),
    threading.Thread(target=print_time, args=('Buzz', 5, 4)),
    threading.Thread(target=print_time, args=('Counter', 1, 20))
]

[t.start() for t in threads]
[t.join() for t in threads]



1 - Counter
2 - Counter
3 - Fizz
3 - Counter
4 - Counter
5 - Buzz
5 - Counter
6 - Fizz
6 - Counter
7 - Counter
8 - Counter
9 - Fizz
9 - Counter
10 - Buzz
10 - Counter
11 - Counter
12 - Fizz
12 - Counter
13 - Counter
14 - Counter
15 - Buzz
15 - Fizz
15 - Counter
16 - Counter
17 - Counter
18 - Fizz
18 - Counter
19 - Counter
20 - Buzz
20 - Counter


[None, None, None]

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import random
import threading
import time

# Recursively find links on a Wikipedia page, 
# then follow a random link, with artificial 5 sec delay
def scrape_article(thread_name, path):
    time.sleep(5)
    print(f'{thread_name}: Scraping {path}')
    html = urlopen('http://en.wikipedia.org{}'.format(path))
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    links = bs.find('div', {'id':'bodyContent'}).find_all('a',
        href=re.compile('^(/wiki/)((?!:).)*$'))
    if len(links) > 0:
        newArticle = links[random.randint(0, len(links)-1)].attrs['href']
        scrape_article(thread_name, newArticle)

           
threads = [
    threading.Thread(target=scrape_article, args=('Thread 1', '/wiki/Kevin_Bacon',)),
    threading.Thread(target=scrape_article, args=('Thread 2', '/wiki/Monty_Python',)),
]
[t.start() for t in threads]
[t.join() for t in threads]

Thread 2: Scraping /wiki/Lifeboat_sketch
Thread 1: Scraping /wiki/Knowledge_Discovery_Metamodel
Thread 1: Scraping /wiki/Pinellas_County,_Florida
Thread 2: Scraping /wiki/Fane_Aircraft_Company
Thread 1: Scraping /wiki/University_Press_of_America
Thread 1: Scraping /wiki/Kevin_Bacon
Thread 2: Scraping /wiki/Monty_Python
Thread 2: Scraping /wiki/List_of_IEC_standards
Thread 2: Scraping /wiki/North_Yorkshire_Moors_Railway
Thread 1: Scraping /wiki/Pacific_Tigers_men%27s_basketball_statistical_leaders
Thread 2: Scraping /wiki/Monty_Python_v._American_Broadcasting_Companies,_Inc.
Thread 1: Scraping /wiki/ISO/IEC_38500


KeyboardInterrupt: 

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import random
import threading
import time


visited = []

def get_links(thread_name, bs):
    print('Getting links in {}'.format(thread_name))
    links = bs.find('div', {'id':'bodyContent'}).find_all('a',
        href=re.compile('^(/wiki/)((?!:).)*$'))
    return [link for link in links if link not in visited]

# Recursively find links on a Wikipedia page, 
# then follow a random link, with artificial 5 sec delay
def scrape_article(thread_name, path):
    time.sleep(5)
    visited.append(path)
    print(f'{thread_name}: Scraping {path}')
    html = urlopen('http://en.wikipedia.org{}'.format(path))
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    links = get_links(thread_name, bs)
    if len(links) > 0:
        newArticle = links[random.randint(0, len(links)-1)].attrs['href']
        scrape_article(thread_name, newArticle)

           
threads = [
    threading.Thread(target=scrape_article, args=('Thread 1', '/wiki/Kevin_Bacon',)),
    threading.Thread(target=scrape_article, args=('Thread 2', '/wiki/Monty_Python',)),
]
[t.start() for t in threads]
[t.join() for t in threads]

Thread 2: Scraping /wiki/List_of_office_suites
Thread 1: Scraping /wiki/Meta-Object_Facility
Thread 1: Scraping /wiki/Cedric_Gibbons
Thread 1: Scraping /wiki/Fujifilm_FinePix_Real_3D
Thread 2: Scraping /wiki/Draft_horse
Thread 2: Scraping /wiki/PDP-9
Thread 2: Scraping /wiki/Sidley_United_F.C.
Thread 1: Scraping /wiki/The_Bulletin_(Bend)
Thread 2: Scraping /wiki/Utagawa_Kunisada_II
Thread 2: Scraping /wiki/Monty_PythonThread 1: Scraping /wiki/Kevin_Bacon

Thread 1: Scraping /wiki/Geographic_coordinate_system
Thread 2: Scraping /wiki/ConceptDraw_Office
Getting links in Thread 1
Getting links in Thread 2
Getting links in Thread 1
Getting links in Thread 2


KeyboardInterrupt: 

In [11]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import random
import threading
from queue import Queue
import time
import pymysql


def storage(queue):
    conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
    user='root', passwd='password', db='mysql', charset='utf8')
    cur = conn.cursor()
    cur.execute('USE wikipedia')
    while 1:
        if not queue.empty():
            path = queue.get()
            cur.execute('SELECT * FROM pages WHERE url = %s', (path))
            if cur.rowcount == 0:
                print(f'Storing article {path}')
                cur.execute('INSERT INTO pages (url) VALUES (%s)', (path))
                conn.commit()
            else:
                print("Article already exists: {}".format(path))

visited = set()
def get_links(thread_name, bs):
    print('Getting links in {}'.format(thread_name))
    links = bs.find('div', {'id':'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))
    links = [link.get('href') for link in links]
    return [link for link in links if link and link not in visited]

def scrape_article(thread_name, path):
    time.sleep(5)
    visited.add(path)
    print(f'{thread_name}: Scraping {path}')
    bs = BeautifulSoup(urlopen('http://en.wikipedia.org{}'.format(path)), 'html.parser')
    links = get_links(thread_name, bs)
    if len(links) > 0:
        [queue.put(link) for link in links]
        newArticle = links[random.randint(0, len(links)-1)].attrs['href']
        scrape_article(thread_name, newArticle)


queue = Queue()


threads = [
    threading.Thread(target=scrape_article, args=('Thread 1', '/wiki/Kevin_Bacon',)),
    threading.Thread(target=scrape_article, args=('Thread 2', '/wiki/Monty_Python',)),
    threading.Thread(target=storage, args=(queue,))
]
[t.start() for t in threads]
[t.join() for t in threads]


Article already exists: /wiki/United_States
Article already exists: /wiki/United_States
Article already exists: /wiki/United_States
Article already exists: /wiki/United_States
Article already exists: /wiki/United_States
Article already exists: /wiki/United_States
Article already exists: /wiki/United_States
Storing article /wiki/1959_in_music
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Billboard_Hot_100
Article already exists: /wiki/Bi

Exception in thread Thread-23:
Traceback (most recent call last):
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/y6/jnf4yrtx1pg3y9tqb8fmhnrr0000gp/T/ipykernel_77749/3538858134.py", line 42, in scrape_article
AttributeError: 'str' object has no attribute 'attrs'
Exception in thread Thread-24:
Traceback (most recent call last):
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner


Article already exists: /wiki/Kevin_Bacon_filmography
Article already exists: /wiki/Kyra_SedgwickStoring article /wiki/List_of_Billboard_number-one_singles_of_1950

Storing article /wiki/Sosie_Bacon
Storing article /wiki/Edmund_Bacon_(architect)Storing article /wiki/List_of_Billboard_number-one_singles_of_1951

Getting links in Thread 2
Storing article /wiki/List_of_Billboard_number-one_singles_of_1952
Storing article /wiki/Michael_Bacon_(musician)Storing article /wiki/List_of_Billboard_number-one_singles_of_1953

Storing article /wiki/List_of_Billboard_number-one_singles_of_1954


    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/y6/jnf4yrtx1pg3y9tqb8fmhnrr0000gp/T/ipykernel_77749/3538858134.py", line 42, in scrape_article
AttributeError: 

Storing article /wiki/Holly_NearStoring article /wiki/List_of_Billboard_number-one_singles_of_1955

Storing article /wiki/List_of_Billboard_number-one_singles_of_1956
Storing article /wiki/Leading_man
Storing article /wiki/List_of_Billboard_number-one_singles_of_1957
Storing article /wiki/Character_actor


'str' object has no attribute 'attrs'


Storing article /wiki/List_of_Billboard_number-one_singles_of_1958
Storing article /wiki/List_of_Billboard_Hot_100_number_ones_of_1960
Storing article /wiki/Golden_Globe_Award
Storing article /wiki/List_of_Billboard_Hot_100_number_ones_of_1961Storing article /wiki/Screen_Actors_Guild_Award

Storing article /wiki/Primetime_Emmy_Award
Storing article /wiki/List_of_Billboard_Hot_100_number_ones_of_1962
Storing article /wiki/List_of_Billboard_Hot_100_number_ones_of_1963Article already exists: /wiki/The_Guardian

Storing article /wiki/List_of_Billboard_Hot_100_number_ones_of_1964Storing article /wiki/Academy_Award

Storing article /wiki/List_of_Billboard_Hot_100_number_ones_of_1965
Storing article /wiki/List_of_Billboard_Hot_100_number_ones_of_1966
Storing article /wiki/Hollywood_Walk_of_FameStoring article /wiki/List_of_Billboard_Hot_100_number_ones_of_1967

Storing article /wiki/List_of_Billboard_Hot_100_number_ones_of_1968
Storing article /wiki/National_Lampoon%27s_Animal_HouseStoring ar

Storing article /wiki/List_of_Your_Hit_Parade_number-one_songs
Storing article /wiki/List_of_Billboard_top-ten_singlesStoring article /wiki/Ellen_Barkin

Storing article /wiki/Billboard_Year-EndArticle already exists: /wiki/Footloose_(1984_film)

Storing article /wiki/James_DeanStoring article /wiki/List_of_Billboard_Hot_100_chart_achievements_and_milestones

Storing article /wiki/Rebel_Without_a_Cause
Article already exists: /wiki/Kevin_Bacon_(disambiguation)
Storing article /wiki/Mickey_RooneyArticle already exists: /wiki/Philadelphia

Article already exists: /wiki/Kevin_Bacon_filmography
Storing article /wiki/Judy_Garland
Article already exists: /wiki/Kyra_Sedgwick
Article already exists: /wiki/Sosie_Bacon
Article already exists: /wiki/Edmund_Bacon_(architect)
Article already exists: /wiki/Michael_Bacon_(musician)Storing article /wiki/Typecasting_(acting)

Article already exists: /wiki/Holly_Near
Storing article /wiki/John_Hughes_(filmmaker)
Article already exists: /wiki/Leading_man

KeyboardInterrupt: 

Article already exists: /wiki/Six_degrees_of_separationStoring article /wiki/Bravo_Otto

Article already exists: /wiki/Internet_meme
Article already exists: /wiki/SixDegrees.org
Article already exists: /wiki/Bravo_Otto
Article already exists: /wiki/Social_networking_service
Article already exists: /wiki/Footloose_(1984_film)
Article already exists: /wiki/Six_Degrees_of_Kevin_Bacon
Article already exists: /wiki/IMDbStoring article /wiki/CableACE_Award

Article already exists: /wiki/Paul_Erd%C5%91sArticle already exists: /wiki/CableACE_Award

Article already exists: /wiki/Losing_ChaseArticle already exists: /wiki/Erd%C5%91s_number

Storing article /wiki/Chlotrudis_Awards
Article already exists: /wiki/Paul_Erd%C5%91s
Article already exists: /wiki/Erd%C5%91s_number
Article already exists: /wiki/The_Woodsman_(2004_film)
Article already exists: /wiki/Erd%C5%91s%E2%80%93Bacon_numberStoring article /wiki/Critics%27_Choice_Movie_Awards



In [12]:
import threading
import time

def print_time(threadName, delay, iterations):
    start = int(time.time())
    for i in range(0,iterations):
        time.sleep(delay)
        seconds_elapsed = str(int(time.time()) - start)
        print ('{} {}'.format(seconds_elapsed, threadName))

threading.Thread(target=print_time, args=('Fizz', 3, 33)).start()
threading.Thread(target=print_time, args=('Buzz', 5, 20)).start()
threading.Thread(target=print_time, args=('Counter', 1, 100)).start()

1 Counter
2 Counter
3 Fizz
3 Counter
4 Counter
5 Buzz
6 Counter
6 Fizz
7 Counter


## Processes

## IMPORTANT NOTE FOR JUPYTER NOTEBOOK

The multiprocessing package does not play nicely with Jupyter Notebooks. Fortunately, there exists the multiprocess package that does work well with it -- and they have the exact same interface. (note: "multiprocessing" vs. "multiprocess" -- two different packages, keep them straight!)
You can install multiprocess, which works with Jupyter notebooks, using:<br>
<code>$ pip install multiprocess</code>

In [7]:
# Uncomment this for non-Notebooks
#from multiprocessing import Process 

# Comment this for non-Notebooks
from multiprocess import Process

import time

def print_time(threadName, delay, iterations):
    start = int(time.time())
    for i in range(0,iterations):
        time.sleep(delay)
        seconds_elapsed = str(int(time.time()) - start)
        print (threadName if threadName else seconds_elapsed)

processes = [
    Process(target=print_time, args=('Counter', 1, 100)),
    Process(target=print_time, args=('Fizz', 3, 33)),
    Process(target=print_time, args=('Buzz', 5, 20)) 
]

[p.start() for p in processes]
[p.join() for p in processes]

Counter
Counter
Fizz
Counter
Counter
Buzz
Counter
Fizz
Counter
Counter
Counter
Fizz
Counter
Buzz
Counter
Counter
Fizz
Counter
Counter
Counter
Buzz
Fizz
Counter
Counter


Process Process-8:
Process Process-6:
Process Process-7:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/y6/jnf4yrtx1pg3y9tqb8f

In [16]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import random

# Uncomment this for non-Notebooks
#from multiprocessing import Process 

# Comment this for non-Notebooks
from multiprocess import Process

import os
import time

visited = []
def get_links(bs):
    links = bs.find('div', {'id':'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))
    return [link for link in links if link not in visited]

def scrape_article(path):
    visited.append(path)
    html = urlopen('http://en.wikipedia.org{}'.format(path))
    time.sleep(5)
    bs = BeautifulSoup(html, 'html.parser')
    print(f'Scraping {bs.find("h1").get_text()} in process {os.getpid()}')
    links = get_links(bs)
    if len(links) > 0:
        scrape_article(links[random.randint(0, len(links)-1)].attrs['href'])


processes = [
    Process(target=scrape_article, args=('/wiki/Kevin_Bacon',)),
    Process(target=scrape_article, args=('/wiki/Monty_Python',)) 
]

[p.start() for p in processes]


[None, None]

Scraping Kevin Bacon in process 4067
Scraping Monty Python in process 4068
Scraping Ewan McGregor in process 4067
Scraping Charisma Records in process 4068
Scraping Renée Zellweger in process 4067
Scraping Genesis (band) in process 4068
Scraping Alana Haim in process 4067
Scraping Maroon 5 in process 4068


Process Process-19:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/y6/jnf4yrtx1pg3y9tqb8fmhnrr0000gp/T/ipykernel_92257/1953871450.py", line 28, in scrape_article
    scrape_article(links[random.randint(0, len(links)-1)].attrs['href'])
  File "/var/folders/y6/jnf4yrtx1pg3y9tqb8fmhnrr0000gp/T/ipykernel_92257/1953871450.py", line 28, in scrape_article
    scrape_articl

In [14]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import random

# Uncomment this for non-Notebooks
#from multiprocessing import Process, Queue

# Comment this for non-Notebooks
from multiprocess import Process, Queue
import os
import time


def task_delegator(taskQueue, urlsQueue):
    #Initialize with a task for each process
    visited = ['/wiki/Kevin_Bacon', '/wiki/Monty_Python']
    taskQueue.put('/wiki/Kevin_Bacon')
    taskQueue.put('/wiki/Monty_Python')

    while 1:
        # Check to see if there are new links in the urlsQueue
        # for processing
        if not urlsQueue.empty():
            links = [link for link in urlsQueue.get() if link not in visited]
            for link in links:
                #Add new link to the taskQueue
                taskQueue.put(link)

def get_links(bs):
    links = bs.find('div', {'id':'bodyContent'}).find_all('a',
        href=re.compile('^(/wiki/)((?!:).)*$'))
    return [link.attrs['href'] for link in links]

def scrape_article(taskQueue, urlsQueue):
    while 1:
        while taskQueue.empty():
            #Sleep 100 ms while waiting for the task queue
            #This should be rare
            time.sleep(.1)
        path = taskQueue.get()
        html = urlopen('http://en.wikipedia.org{}'.format(path))
        time.sleep(5)
        bs = BeautifulSoup(html, 'html.parser')
        print(f'Scraping {bs.find('h1').get_text()} in process {os.getpid()}')
        links = get_links(bs)
        #Send these to the delegator for processing
        urlsQueue.put(links)

taskQueue = Queue()
urlsQueue = Queue()

processes = [
    Process(target=task_delegator, args=(taskQueue, urlsQueue,)),
    Process(target=scrape_article, args=(taskQueue, urlsQueue,)),
    Process(target=scrape_article, args=(taskQueue, urlsQueue,))
]

[p.start() for p in processes]
    

[None, None, None]

Scraping Kevin Bacon in process 97023
Scraping Monty Python in process 97024
Scraping Kevin Bacon (disambiguation) in process 97023
Scraping Philadelphia in process 97024
Scraping Kevin Bacon filmography in process 97023
Scraping Kyra Sedgwick in process 97024
Scraping Sosie Bacon in process 97023
Scraping Edmund Bacon (architect) in process 97024
Scraping Michael Bacon (musician) in process 97023
Scraping Holly Near in process 97024
Scraping Leading actor in process 97023


Process Process-17:
Process Process-16:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/RSpecht/opt/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/y6/jnf4yrtx1pg3y9tqb8fmhnrr0000gp/T/ipykernel_92257/3812966439.py", line 43, in scrape_article
    time.sleep(5)
  File "/var/folders/y6/jnf4yrtx1pg3y9tqb8fmhnrr0000gp/T/ipykernel_92257/3812966439.py", line 43, in scrape_article
    time.sleep(5)
KeyboardInterrupt
KeyboardInterrupt
Process Process-15